<a href="https://colab.research.google.com/github/Heba2424/DEPI_Projects/blob/main/Arabic_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers datasets torch sklearn


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [ ]:

# Load the Arabic sentiment dataset from Hugging Face
dataset = load_dataset("sepidmnorozy/Arabic_sentiment")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 2468
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 353
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 706
    })
})

In [ ]:

# Load pre-trained Arabic BERT model (AraBERT) and tokenizer
model_name = "aubmindlab/bert-base-arabertv02"  # This is AraBERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: positive, neutral, negative

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Freeze BERT layers (all layers except the classifier)
for param in model.bert.parameters():
    param.requires_grad = False

In [ ]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [ ]:
# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/353 [00:00<?, ? examples/s]

In [ ]:
# Split the dataset into train and test sets
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

In [ ]:
# Define the metrics for evaluation
def compute_metrics(p):
    preds = torch.argmax(torch.Tensor(p.predictions), axis=1) # Convert p.predictions to a PyTorch tensor
    accuracy = accuracy_score(p.label_ids, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.875735,0.633144,0.651086,0.633144,0.641665
2,No log,0.739819,0.691218,0.697143,0.691218,0.694168
3,No log,0.713284,0.698300,0.702344,0.698300,0.700281


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=465, training_loss=0.8771615633400538, metrics={'train_runtime': 325.4067, 'train_samples_per_second': 22.753, 'train_steps_per_second': 1.429, 'total_flos': 1948091744858112.0, 'train_loss': 0.8771615633400538, 'epoch': 3.0})

In [ ]:
#Evaluate the model
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.7132844924926758,
 'eval_accuracy': 0.6983002832861189,
 'eval_precision': 0.7023442712791584,
 'eval_recall': 0.6983002832861189,
 'eval_f1': 0.7002810740848802,
 'eval_runtime': 21.1824,
 'eval_samples_per_second': 33.329,
 'eval_steps_per_second': 2.124,
 'epoch': 3.0}

In [ ]:
# # Fine-tune the model without freezing
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.254528,0.924929,0.926142,0.924929,0.924863
2,No log,0.252895,0.933428,0.933837,0.933428,0.933406
3,No log,0.311021,0.915014,0.915073,0.915014,0.915014


TrainOutput(global_step=465, training_loss=0.09962222191595262, metrics={'train_runtime': 770.7322, 'train_samples_per_second': 9.606, 'train_steps_per_second': 0.603, 'total_flos': 1948091744858112.0, 'train_loss': 0.09962222191595262, 'epoch': 3.0})

In [ ]:
# # Evaluate the model without freezing
# trainer.evaluate()

{'eval_loss': 0.31102126836776733,
 'eval_accuracy': 0.9150141643059491,
 'eval_precision': 0.9150729108515486,
 'eval_recall': 0.9150141643059491,
 'eval_f1': 0.9150141643059491,
 'eval_runtime': 23.0723,
 'eval_samples_per_second': 30.599,
 'eval_steps_per_second': 1.95,
 'epoch': 3.0}

In [ ]:
# Function to get sentence embeddings using BERT (CLS token embedding)
def get_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Add this line

    with torch.no_grad(): # Disable gradient calculation for inference
        outputs = model(**inputs)
    # Access the logits for classification tasks
    cls_embedding = outputs.logits.detach().cpu().numpy()  # Move the tensor to CPU before converting to NumPy
    return cls_embedding

In [ ]:
# Prepare dataset: Get embeddings and labels
X = []  # Sentence embeddings
y = []  # Labels

for example in dataset['train']:
    embedding = get_sentence_embedding(example['text'])
    X.append(embedding.flatten())  # Flatten to make it 1D
    y.append(example['label'])

X = np.array(X)
y = np.array(y)

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train a classifier (Logistic Regression in this case)
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# Evaluate the classifier
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

Accuracy: 0.7348
Precision: 0.7357, Recall: 0.7348, F1: 0.7347


In [ ]:
# Function to compute cosine similarity
def get_most_similar_sentiment(input_text, X, y):
    input_embedding = get_sentence_embedding(input_text).flatten().reshape(1, -1)  # Get embedding for input text
    similarities = cosine_similarity(input_embedding, X)  # Compute cosine similarity
    most_similar_index = np.argmax(similarities)  # Get the index of the most similar sentence
    return y[most_similar_index]  # Return the sentiment label of the most similar sentence

In [ ]:
# Function to compute cosine similarity and find the most similar sentence
def get_most_similar_sentiment(input_text, X_train_embeddings, y_train_labels):
    input_embedding = get_sentence_embedding(input_text).reshape(1, -1)  # Get embedding for input text
    similarities = cosine_similarity(input_embedding, X_train_embeddings)  # Compute cosine similarity
    most_similar_index = np.argmax(similarities)  # Get the index of the most similar sentence
    return y_train_labels[most_similar_index]  # Return the sentiment label of the most similar sentence

In [ ]:
# Test the cosine similarity approach on the test set
y_pred_cosine = []

for test_example in dataset['test']:
    text = test_example['text']
    true_label = test_example['label']

    # Predict sentiment using cosine similarity
    predicted_label = get_most_similar_sentiment(text, X_train, y_train)
    y_pred_cosine.append(predicted_label)

# Get the true labels from the test set
y_true = [example['label'] for example in dataset['test']]

# Calculate accuracy
accuracy_cosine = accuracy_score(y_true, y_pred_cosine)
print(f"Cosine Similarity Accuracy: {accuracy_cosine:.4f}")


Cosine Similarity Accuracy: 0.6246


In [ ]:
# Test the cosine similarity method with a new sentence
new_text = "أحببت هذا الفيلم كثيراً"  # Example Arabic sentence
predicted_sentiment = get_most_similar_sentiment(new_text, X_train, y_train)

print(f"Predicted sentiment for the input text: {predicted_sentiment}")

Predicted sentiment for the input text: 0


In [ ]:
# Test the cosine similarity method with a new sentence
new_text = " هذا الفيلم رائع جدا جدا وممتع جدا"  # Example Arabic sentence
predicted_sentiment = get_most_similar_sentiment(new_text, X_train, y_train)

print(f"Predicted sentiment for the input text: {predicted_sentiment}")

Predicted sentiment for the input text: 1


In [ ]:
# Test the cosine similarity method with a new sentence
new_text = "هذا الفيلم سيئ جدا جدا وممل"  # Example Arabic sentence
predicted_sentiment = get_most_similar_sentiment(new_text, X_train, y_train)

print(f"Predicted sentiment for the input text: {predicted_sentiment}")

Predicted sentiment for the input text: 0


In [ ]:
# Save the model and tokenizer
model.save_pretrained("./arabic-sentiment-bert")
tokenizer.save_pretrained("./arabic-sentiment-bert")

('./arabic-sentiment-bert/tokenizer_config.json',
 './arabic-sentiment-bert/special_tokens_map.json',
 './arabic-sentiment-bert/vocab.txt',
 './arabic-sentiment-bert/added_tokens.json',
 './arabic-sentiment-bert/tokenizer.json')